In [302]:
from data_loading_fun_helper import read_data
from data_loading_fun_helper import create_dataset

from text_preprocessing_helper import cleaning_text
from text_preprocessing_helper import check_spelling
from text_preprocessing_helper import remove_stopwords
from text_preprocessing_helper import lem_tokens
from text_preprocessing_helper import find_new_answer_indexes

from word_embedding.CosineSimilarityConv import CosineSimilarityConv
from model import RNN

import nltk
from collections import Counter
import pandas as pd
import numpy as np
import os
import string
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import json

In [260]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU:', torch.cuda.device_count())
    print('Device name', torch.cuda.get_device_name(0))
else:
    print('No GPU, using CPU')
    device = torch.device('cpu')

No GPU, using CPU


In [261]:
# Путь ./dataset/train.json указывает на файл train.json в подкаталоге dataset, который находится в текущем 
# рабочем каталоге. Точка . означает текущий каталог, поэтому ./ можно опустить, и просто написать 
# dataset/train.json.

train = './dataset/train.json'
test = './dataset/test.json'

In [262]:
train = read_data(train)
train = create_dataset(data = train, label = "train")
test = read_data(test)
test = create_dataset(data = test, label = "test")
general_data_list = {'train': train, 'test': test}

In [263]:
# наиболее часто встречающиеся слова в фрагменте текста из поля `text`, соответствующий пункту анкеты
enforcement_contract_count = train[train['label']=='обеспечение исполнения контракта']\
                                        ['extracted_text'].str.split().explode().value_counts()
enforcement_contract = pd.DataFrame(enforcement_contract_count)

enforcement_contract['extracted_text'] = (enforcement_contract['extracted_text'] / 
                                          enforcement_contract.sum(axis = 0).values[0]) * 100
enforcement_contract = enforcement_contract[enforcement_contract['extracted_text']>1]

In [264]:
barplot = go.Bar(
    x = enforcement_contract.index,
    y = enforcement_contract.extracted_text,
    text = enforcement_contract.index,
    textposition='auto',
)

# Создание объекта Layout для настройки графика
layout = go.Layout(
    title = 'Частота встречаимости слов в фрагменте текста из поля `text`, соответствующий пункту анкеты',
    xaxis=dict(title='Слово'),
    yaxis=dict(title='Количество всречаимости слова') 
)

# Создание объекта Figure и добавление на него barplot и layout
fig = go.Figure(data=[barplot], layout=layout)

# Отображение графика
fig.show()

Заметим что у нас есть такие пары как "обеспечения" - "обеспечение" - "Обеспечение" или "Размер" - "размере". В последствии произведем лемматизацию чтобы привести все слова к одной форме

In [265]:
def build_peplane_text_processing():
    for label, df in general_data_list.items():
        if label == 'train':
            cleaning_text(df, "text")
            cleaning_text(df, "extracted_text")
            check_spelling(df, label)
            remove_stopwords(df, label)
            lem_tokens(df, label)
            find_new_answer_indexes(df)
            
            
        elif label == 'test':
            cleaning_text(df, "text")
            check_spelling(df, label)
            remove_stopwords(df, label)
            lem_tokens(df, label)
        

Запустим предобработку данных, очистка текста, проверка орфографии, удаление стоп лов, лемматизация

In [266]:
build_peplane_text_processing()

In [267]:
def count_words(sentence):
    # Среднее количество слов в extracted_text колонке
    return len(sentence.split())

avg_words_per_sentence = train['extracted_text'].apply(count_words).mean()
print(avg_words_per_sentence)

8.156197887715397


Какие комбинации из 8ми слов чаще всего встречаются в extracted_text

In [268]:
text = ' '.join(train['extracted_text'])
tokens = nltk.word_tokenize(text)
karnel_list = []

# Генерация n-грамм
ngrams = nltk.ngrams(tokens, 8) 
# Подсчет количества вхождений каждой комбинации
ngrams_counter = Counter(ngrams)

# Вывод 50 наиболее часто встречающихся комбинаций --> именно они и будут нашими ядрами
for counter_most in ngrams_counter.most_common(50):
    karnel_text = ' '.join(counter_most[0])
    karnel_list.append(karnel_text)

Разделим на train и valid датафреймы 

In [271]:
TEST_SIZE = 0.2
SEED = 42

train_df, valid_df = train_test_split(train, test_size = TEST_SIZE, random_state = SEED)

Получим метки answer_start и answer_end

In [272]:
label_train = train_df[['answer_start', 'answer_end']].values
label_train = torch.tensor(label_train)

label_valid = valid_df[['answer_start', 'answer_end']].values
label_valid = torch.tensor(label_valid)

Преобразуем текст в векторную форму

In [274]:
cosine_conv = CosineSimilarityConv(train_df, valid_df, test, 'text')

In [275]:
# У нас 50 ядер
cosine_conv.create_kernel_map(karnel_list, stride = 8, label = 'train')
train_df = cosine_conv.sum_all_cosine_map()

  0%|          | 0/50 [00:00<?, ?it/s]

In [283]:
cosine_conv.create_kernel_map(karnel_list, stride = 8, label = 'valid')
valid_df = cosine_conv.sum_all_cosine_map()

  0%|          | 0/50 [00:00<?, ?it/s]

In [284]:
cosine_conv.create_kernel_map(karnel_list, stride = 8, label = 'test')
test_df = cosine_conv.sum_all_cosine_map()

  0%|          | 0/50 [00:00<?, ?it/s]

Дополним valid_df и test_df до размерности train_df

In [285]:
valid_df = np.pad(valid_df, ((0,0),(0,20)), mode='constant')
test_df = np.pad(test_df, ((0,0),(0,17)), mode='constant')

In [287]:
print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)

(1439, 51)
(360, 51)
(318, 51)


Загрузим данные в DataLoader

In [288]:
BATCH_SIZE = 32

train_dataset = TensorDataset(torch.tensor(train_df), label_train)
train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE)

valid_dataset = TensorDataset(torch.tensor(valid_df), label_valid)
valid_dataloader = DataLoader(valid_dataset, batch_size = BATCH_SIZE)

Модель

In [289]:
# путь для сохранения модели
model_path = './save/model.ckpt'  

In [290]:
# создание модели
input_size = 51
hidden_size = 64
num_layers = 2
output_size = 2
learning_rate = 0.001
num_epochs = 150
lambda_l2 = 0.01  # коэффициент L2 регуляризации

In [291]:
model = RNN(input_size, hidden_size, num_layers, output_size)
model.to(device) 
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lambda_l2)

In [292]:
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_dataloader):
            
        inputs = torch.tensor(inputs, dtype=torch.float).to(device)
        labels = torch.tensor(labels, dtype=torch.float).to(device)
        
        # Forward pass
        outputs_start, outputs_end = model(inputs)
        labels_start, labels_end = labels[:, 0], labels[:, 1] 
        
        loss_start = criterion(outputs_start, labels_start)
        loss_end = criterion(outputs_end, labels_end)
        loss = loss_start + loss_end
        
        # обнгуляем градиент, шаг backpropagation, шаг оптимизатора
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        
        
    print('Epoch [{}/{}], Loss: {:.4f}'.format(
        epoch+1, num_epochs, loss.item())) 

/tmp/ipykernel_632171/3239672180.py:4: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/tmp/ipykernel_632171/3239672180.py:5: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch [1/150], Loss: 340247.9375
Epoch [2/150], Loss: 27159.6523
Epoch [3/150], Loss: 12353.2070
Epoch [4/150], Loss: 13331.3242
Epoch [5/150], Loss: 12685.4395
Epoch [6/150], Loss: 10468.8223
Epoch [7/150], Loss: 9325.1299
Epoch [8/150], Loss: 8817.0986
Epoch [9/150], Loss: 8182.8506
Epoch [10/150], Loss: 7505.8115
Epoch [11/150], Loss: 7005.2803
Epoch [12/150], Loss: 6654.2041
Epoch [13/150], Loss: 5827.8379
Epoch [14/150], Loss: 5302.2988
Epoch [15/150], Loss: 4573.8462
Epoch [16/150], Loss: 3997.1089
Epoch [17/150], Loss: 4138.7695
Epoch [18/150], Loss: 5940.0879
Epoch [19/150], Loss: 27972.0039
Epoch [20/150], Loss: 22066.2422
Epoch [21/150], Loss: 5182.5723
Epoch [22/150], Loss: 5832.0186
Epoch [23/150], Loss: 4571.3101
Epoch [24/150], Loss: 4844.2261
Epoch [25/150], Loss: 4688.5820
Epoch [26/150], Loss: 4366.2593
Epoch [27/150], Loss: 4280.2510
Epoch [28/150], Loss: 4218.2627
Epoch [29/150], Loss: 4083.7261
Epoch [30/150], Loss: 4272.9116
Epoch [31/150], Loss: 4312.6792
Epoch [3

In [293]:
# Сохранение модели
torch.save(model.state_dict(), model_path)

In [294]:
# Если есть сохраненные веса используем для предсказания их
if os.path.exists(model_path):
    model = RNN(input_size, hidden_size, num_layers, output_size)
    model.to(device) 
    model.load_state_dict(torch.load(model_path))

In [295]:
pred_data = pd.DataFrame(data=[], columns=['answer_start', 'answer_end'])

In [296]:
# Предсказание данных
with torch.no_grad():
    for ind, inputs in enumerate(test_df):
       
        inputs = torch.tensor(inputs, dtype=torch.float).to(device)
        inputs = inputs.unsqueeze(0)
        outputs = model.forward(inputs)
        start_ind, end_ind = outputs
        start_ind = torch.round(start_ind)
        end_ind = torch.round(end_ind)
        answer_list = sorted([int(start_ind), int(end_ind)])
        pred_data.loc[ind, 'answer_start'] = answer_list[0]
        pred_data.loc[ind, 'answer_end'] = answer_list[1]


In [297]:
test = pd.concat([test, pred_data], axis=1)

In [298]:
for ind, row in test.iterrows():
    answer_start = row.answer_start
    answer_end = row.answer_end
    text = row.text
    text = list(text)
    text = text[answer_start:answer_end]
    text = ''.join(text)
    test.loc[ind, 'extracted_text'] = text

In [299]:
test.head()

,id,text,label,answer_start,answer_end,extracted_text
0,762883279,муниципальный контракт оказание услуга техниче...,обеспечение исполнения контракта,984,1061,оставляться заказчик заключение контракт разме...
1,311837655,извещение проведение электронный аукцион закуп...,обеспечение исполнения контракта,1266,1365,ск номер корреспондентский счёт требование гар...
2,540954893,идентификационный код закупка контракт поставк...,обеспечение исполнения контракта,1329,1432,печение государственный муниципальный нужда об...
3,274660397,идентификационный код закупка контракт поставк...,обеспечение исполнения контракта,1322,1426,а услуга обеспечение государственный муниципал...
4,732742591,идентификационный код закупка контракт поставк...,обеспечение исполнения контракта,1336,1441,уга обеспечение государственный муниципальный ...


Преобразуем в json

In [304]:
result_dict = []

for _, row in test.iterrows():
    
    extracted_part = {
        'text': row['extracted_text'],
        'answer_start': row['answer_start'],
        'answer_end': row['answer_end']
    }
    
    result_dict.append({
        'id': row['id'],
        'text': row['text'],
        'label': row['label'],
        'extracted_part': extracted_part
    })

# Сохранение словаря в файл в формате JSON
with open('predictions.json', 'w') as f:
    json.dump(result_dict, f)